# Countries Pipeline(v1)

## TODO

1. [전 세계 국가에 대한 정보를 제공하는 API](https://restcountries.com/#rest-countries)로부터 데이터를 수집하여 필요한 필드만 추출하고, Redshift에 적재한다.
2. 기존 테이블의 모든 데이터를 지우고, 새 데이터를 처음부터 다시 채우는 방법인 Full Refresh 방법으로 구현한다.
3. 간단한 프로토타입을 만드는 것을 목표이기 때문에 최소한의 작업만 진행한다.

In [3]:
!pip install -r ../requirements.txt > install.log 2>&1

In [2]:
import requests
import pprint
from dotenv import load_dotenv
import os
import psycopg2

In [3]:
load_dotenv()

host = os.getenv("REDSHIFT_HOST")
port = os.getenv("REDSHIFT_PORT")
dbname = os.getenv("REDSHIFT_DBNAME")
user = os.getenv("REDSHIFT_USER")
password = os.getenv("REDSHIFT_PASSWORD")

In [4]:
link = "https://restcountries.com/v3/all"

response = requests.get(link)
records = response.json()

In [5]:
sample_record = records[0]

pprint.pp(sample_record)

{'name': {'common': 'Gibraltar',
          'official': 'Gibraltar',
          'nativeName': {'eng': {'official': 'Gibraltar',
                                 'common': 'Gibraltar'}}},
 'tld': ['.gi'],
 'cca2': 'GI',
 'ccn3': '292',
 'cca3': 'GIB',
 'independent': False,
 'status': 'officially-assigned',
 'unMember': False,
 'currencies': {'GIP': {'name': 'Gibraltar pound', 'symbol': '£'}},
 'idd': {'root': '+3', 'suffixes': ['50']},
 'capital': ['Gibraltar'],
 'altSpellings': ['GI'],
 'region': 'Europe',
 'subregion': 'Southern Europe',
 'languages': {'eng': 'English'},
 'translations': {'ara': {'official': 'جبل طارق', 'common': 'جبل طارق'},
                  'bre': {'official': 'Jibraltar', 'common': 'Jibraltar'},
                  'ces': {'official': 'Gibraltar', 'common': 'Gibraltar'},
                  'cym': {'official': 'Gibraltar', 'common': 'Gibraltar'},
                  'deu': {'official': 'Gibraltar', 'common': 'Gibraltar'},
                  'est': {'official': 'Gibraltar'

In [6]:
country = sample_record["name"]["official"]

population = sample_record["population"]

area = sample_record["area"]

print(f"""country: {country}
population: {population}
area: {area}""")

country: Gibraltar
population: 33691
area: 6.0


In [7]:
conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)
conn.autocommit = True

cur = conn.cursor()

drop_table_query = """
DROP TABLE IF EXISTS dmhuh1003.country;
"""

cur.execute(drop_table_query)

create_table_query = """
CREATE TABLE IF NOT EXISTS dmhuh1003.country (
    name VARCHAR(128),
    population INTEGER,
    area REAL
);
"""

cur.execute(create_table_query)

In [ ]:
for record in records:
    country = record["name"]["official"]
    population = record["population"]
    area = record["area"]

    data = (country, population, area)

    insert_query = """
    INSERT INTO dmhuh1003.country (name, population, area)
    VALUES (%s, %s, %s);
    """

    cur.execute(insert_query, data)

## Next Steps

1. 기능별 코드 분리

    지금 코드는 하나의 스크립트 형태로 작성되어 있어서 전체 흐름을 파악하기가 어렵다. 주요 기능을 함수로 분리하면 가독성도 좋아지고, 코드 구조도 명확해질 것 같다.

2. 트랜잭션 적용
   
   여기서 테이블을 삭제하고, 생성하고, 데이터를 넣는 작업은 하나의 논리적 단위를 이루는 과정이다. 하지만 현재는 각 쿼리가 개별적으로 실행되기 때문에, 중간에 오류가 발생하면 데이터의 일관성이 깨질 위험이 있다. 오류가 생기면 전체 작업을 롤백하여 데이터를 무결성을 유지할 수 있도록 트랜잭션을 적용하는 게 좋겠다.

3. 로깅 추가

    지금 코드는 실행 중에 어떤 작업이 진행되고 있는지 파악하기가 어렵다. 각 단계에서 진행 상황과 오류를 로깅하면 디버깅이나 유지 보수가 훨씬 수월해질 것 같다.